## stack, melt로 넓은 데이터를 긴 포맷으로 리셰이핑
- 열 이름에 변숫값이 포함되는 형태가 많음
- NLS 데이터를 타이딩 한다는 것 -> weeksworked00 ~ weeksworked04까지의 열을 1인당 1년간 단 하나의 근무 주 수 열과 연도 열을 가진 다섯 개의 행으로 변환하는 것

In [5]:
import pandas as pd

nls97=pd.read_csv('C:/data-cleansing-main/Chapter09/data/nls97f.csv')

In [8]:
nls97.set_index(['originalid'],inplace=True)

weeksworkedcols = ['weeksworked00','weeksworked01','weeksworked02','weeksworked03','weeksworked04']

nls97[weeksworkedcols].head(2).T
nls97.shape

(8984, 89)

In [10]:
nls97[weeksworkedcols]

,weeksworked00,weeksworked01,weeksworked02,weeksworked03,weeksworked04
originalid,,,,,
8245,46.0,52.0,52.0,48.0,52.0
3962,5.0,49.0,52.0,52.0,52.0
3571,5.0,8.0,52.0,35.0,40.0
2979,29.0,34.0,52.0,31.0,52.0
8511,53.0,52.0,52.0,52.0,48.0
...,...,...,...,...,...
7217,27.0,52.0,52.0,33.0,15.0
2,51.0,52.0,44.0,45.0,52.0
5113,0.0,0.0,20.0,23.0,8.0


### stack을 사용해 넓은 데이터를 긴 데이터로 변환
- 원래 데이터프레임의 열 이름을 인덱스로 옮김

In [11]:
weeksworked=nls97[weeksworkedcols].stack(dropna=False).reset_index().rename(columns={'level_1':'year',0:'weeksworked'})

weeksworked.head(10)

,originalid,year,weeksworked
0,8245,weeksworked00,46.0
1,8245,weeksworked01,52.0
2,8245,weeksworked02,52.0
3,8245,weeksworked03,48.0
4,8245,weeksworked04,52.0
5,3962,weeksworked00,5.0
6,3962,weeksworked01,49.0
7,3962,weeksworked02,52.0
8,3962,weeksworked03,52.0
9,3962,weeksworked04,52.0


In [12]:
# year값 수정
weeksworked['year']=weeksworked.year.str[-2:].astype(int)+2000
weeksworked.head(10)

,originalid,year,weeksworked
0,8245,2000,46.0
1,8245,2001,52.0
2,8245,2002,52.0
3,8245,2003,48.0
4,8245,2004,52.0
5,3962,2000,5.0
6,3962,2001,49.0
7,3962,2002,52.0
8,3962,2003,52.0
9,3962,2004,52.0


### melt 사용 - 사용법 잘 모르겠음..

In [15]:
weeksworked=nls97.reset_index().loc[:,['originalid']+weeksworkedcols].melt(id_vars=['originalid'],value_vars=weeksworkedcols,var_name='year',value_name='weeksworked')
# id_vars: 기준이 되는 컬럼 지정 / value_vars: 녹여서 값과 같이 행으로 들어갈 컬럼
weeksworked['year']=weeksworked.year.str[-2:].astype(int)+2000
weeksworked.set_index(['originalid'],inplace=True)
weeksworked.loc[[8245,3962]]

,year,weeksworked
originalid,,
8245,2000,46.0
8245,2001,52.0
8245,2002,52.0
8245,2003,48.0
8245,2004,52.0
3962,2000,5.0
3962,2001,49.0
3962,2002,52.0
3962,2003,52.0


In [16]:
# 대학 등록 열을 녹여서 리셰이핑
colenrcols = ['colenroct00','colenroct01','colenroct02','colenroct03','colenroct04']
colenr = nls97.reset_index().loc[:,['originalid'] + colenrcols].melt(id_vars=['originalid'], value_vars=colenrcols, var_name='year', value_name='colenr')

colenr['year'] = colenr.year.str[-2:].astype(int)+2000
colenr.set_index(['originalid'], inplace=True)
colenr.loc[[8245,3962]]

,year,colenr
originalid,,
8245,2000,1. Not enrolled
8245,2001,1. Not enrolled
8245,2002,1. Not enrolled
8245,2003,1. Not enrolled
8245,2004,1. Not enrolled
3962,2000,1. Not enrolled
3962,2001,1. Not enrolled
3962,2002,1. Not enrolled
3962,2003,1. Not enrolled


In [19]:
# 근무 주 수와 대학 등록 데이터 병합
workschool=pd.merge(weeksworked,colenr,on=['originalid','year'],how='inner')
workschool.loc[[8245,3962]]

,year,weeksworked,colenr
originalid,,,
8245,2000,46.0,1. Not enrolled
8245,2001,52.0,1. Not enrolled
8245,2002,52.0,1. Not enrolled
8245,2003,48.0,1. Not enrolled
8245,2004,52.0,1. Not enrolled
3962,2000,5.0,1. Not enrolled
3962,2001,49.0,1. Not enrolled
3962,2002,52.0,1. Not enrolled
3962,2003,52.0,1. Not enrolled


## 정리
1. stack과 melt 중 melt가 더 유연하다.